In [1]:
import sys
# adding to the path variables the one folder higher (locally, not changing system variables)
sys.path.append("..")
import pandas as pd
import numpy as np
import warnings
import mlflow
from modeling.config import TRACKING_URI, EXPERIMENT_NAME
from pandas_profiling import ProfileReport
import pyarrow.feather as feather
import time
from datetime import datetime


In [2]:
export_df = feather.read_feather("../data/cleaned_data.feather")

In [3]:
export_df["event_timestamp"]=pd.to_datetime(export_df["event_timestamp"]*1000, unit="ns", utc=True)

In [4]:
export_df["notif_viewed_ontime"]=list(map(lambda x,y,z: '' if x=="notification_received" else (1 if y < z else 0), export_df["event_name"], export_df["event_timestamp"],export_df["closed_at"])) #1 if the user view it on time  0 if not  and null if hasnt yet seen it 

In [5]:
export_df['notif_reaction'] = np.where(export_df["notif_viewed_ontime"] == '', 0, 1) #0 if no reaction 1 if have reacted  
export_df.drop(export_df[export_df.notif_viewed_ontime ==''].index , inplace=True)
export_df.reset_index(drop=True, inplace=True)

In [6]:
export_df["reaction_time"]=list(map(lambda x,y,z,w: y-z
if (x==1 and y>z) else (y-w if (x==1 and y>w) else 0), 
export_df["notif_viewed_ontime"],export_df["event_timestamp"],export_df["published_at"],export_df["created_at"]))

In [7]:
active_df=export_df[export_df["notif_viewed_ontime"]==1].reset_index(drop=True)
unactive_df=export_df[export_df["notif_viewed_ontime"]==0].reset_index(drop=True)
active_df.drop('notif_viewed_ontime', axis=1, inplace=True)
unactive_df.drop('notif_viewed_ontime', axis=1, inplace=True)
pd.set_option('display.max_columns', None)
unactive_df


,event_timestamp,event_name,user_id,document_id,surrogate_id,created_at,published_at,closed_at,cause,effect,description,area_of_effect_coordinates,agency_GewRJAw5tUmC4Ku4AX1-SQ,agency_GtvOEQAFZ0GtU6u4AXwvPg,agency_HE59N3RXM0q5vKu4AXlQZg,agency_JUR9bFXmVkWDHqu4AXaY0g,agency_JfA8Bw8Zp024Kqu4AXiSpQ,agency_MgUq5b9mOEunx6u4AXt_BA,agency_NuuRQ2I1Q0a50Kv-AVKlLA,agency_V2AIQQKgmUO3VazvAOA-Cw,agency_jLjibFoim0iwWau4AWoEdQ,agency_pky7jovXYkaw-awAAMrQ3g,agency_zCy9zG00HEqGeKu4AWZYNQ,notif_reaction,reaction_time
0,2021-05-29 04:16:36.618000+00:00,notification_dismiss,8c7b7,-Maps-egC8fFLZM4HR5g,wB4ikyLA6xGpKOFIXMTiIg,2021-05-29 02:07:09+00:00,2021-05-29 02:07:12+00:00,2021-05-29 03:01:00+00:00,VehicularAccident,Delays,⌛ ¡No de nuevo! Hay retrasos por accidente ve...,"[{'latitude': -99.17095221364039, 'longitude':...",0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0
1,2021-05-29 04:16:36.622001+00:00,notification_dismiss,8c7b7,-Maq7-D7bhcEaFBU1LDW,gDALVizA6xGpKOFIXMTiIg,2021-05-29 03:17:01+00:00,2021-05-29 03:17:12+00:00,2021-05-29 04:01:00+00:00,Incident,Delays,⏳ ¡Pa variar! Usuarios reportan retrasos de ha...,"[{'latitude': None, 'longitude': None}]",0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0
2,2021-05-29 17:14:09.994000+00:00,notification_dismiss,8c7b7,-MasFHCt_lTuNt_x16dH,oA37g3_A6xGNq1u4NrDE6w,2021-05-29 13:12:27+00:00,2021-05-29 13:12:42+00:00,2021-05-29 14:01:00+00:00,Incident,Delays,⏳ ¡No te esponjes! Usuarios reportan retrasos ...,"[{'latitude': None, 'longitude': None}]",0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0
3,2021-05-29 03:17:06.023000+00:00,notification_dismiss,3f1e8,-MaphUnb8wb5IgmR7X9E,gGZoKBzA6xGpKOFIXMTiIg,2021-05-29 01:21:13+00:00,2021-05-29 01:21:34+00:00,2021-05-29 02:17:00+00:00,BrokenVehicle,Delays,🚗 ¡Sopas! Retrasos por vehículo descompuesto e...,"[{'latitude': -99.06686958889327, 'longitude':...",0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1,0
4,2021-05-29 01:27:23.251000+00:00,notification_dismiss,81354,-MaogZKKMIQ_Virmn0eB,UIg1h_S_6xGvWA3t6Gmi5Q,2021-05-28 20:37:32+00:00,2021-05-28 20:38:12+00:00,2021-05-28 21:03:00+00:00,VehicularAccident,Delays,⌛ ¡Ah jijo! Hay retrasos por accidente vehicul...,"[{'latitude': -99.08261816451139, 'longitude':...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
389878,2021-07-06 12:16:54.144002+00:00,notification_opened,b6c41,-MdvT00Ih682ZROXWv-n,MOp-FUne6xGEdyPHk--PKA,2021-07-06 10:58:23+00:00,2021-07-06 10:58:26+00:00,2021-07-06 12:00:00+00:00,Incident,Delays,⏳ ¡Changos! Usuarios informan retrasos de más ...,"[{'latitude': None, 'longitude': None}]",0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0
389879,2021-07-06 13:04:46.270002+00:00,notification_opened,b6c41,-MdvkXIakRmEyb5NrwhW,YJhCY1Te6xGEdyPHk--PKA,2021-07-06 12:19:18+00:00,2021-07-06 12:19:25+00:00,2021-07-06 12:50:00+00:00,VehicularAccident,LaneReduction,⏳ ¡Chanfle! Reducción de carriles por percance...,"[{'latitude': -99.05934617140683, 'longitude':...",0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1,0
389880,2021-07-05 23:05:16.648000+00:00,notification_opened,7a79c,-Mdsb6FPBr0BBEK8UFBY,oHf4ctnd6xGfY3t29ZPLVg,2021-07-05 21:39:17+00:00,2021-07-05 21:39:20+00:00,2021-07-05 22:40:01+00:00,Incident,Delays,"⏳ No te agüites, pero usuarios reportan retras...","[{'latitude': None, 'longitude': None}]",0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0
389881,2021-07-06 17:08:21.329002+00:00,notification_opened,0bf97,-Mdwhnz9Hyl6NGRfhlhM,oAKtyXne6xGa3QeEZ-Zg-g,2021-07-06 16:47:02+00:00,2021-07-06 16:47:39+00:00,NaT,Demonstration,Delays,✋ ¡Chanclas! Hay cierre de circulación por man...,"[{'latitude': -99.04188012969783, 'longitude':...",0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1,0


In [8]:
active_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 526984 entries, 0 to 526983
Data columns (total 25 columns):
 #   Column                         Non-Null Count   Dtype              
---  ------                         --------------   -----              
 0   event_timestamp                526984 non-null  datetime64[ns, UTC]
 1   event_name                     526984 non-null  object             
 2   user_id                        526984 non-null  object             
 3   document_id                    526984 non-null  object             
 4   surrogate_id                   526984 non-null  object             
 5   created_at                     526984 non-null  datetime64[ns, UTC]
 6   published_at                   526984 non-null  datetime64[ns, UTC]
 7   closed_at                      526984 non-null  datetime64[ns, UTC]
 8   cause                          526984 non-null  object             
 9   effect                         526984 non-null  object             
 10  descript

In [9]:
active_df.loc[active_df['event_name'].isin(['notification_opened', 'notification_view_alternatives' ,  'notification_share']),'opened']= 1
active_df.loc[active_df['event_name'].isin(['notification_received', 'notification_dismiss']),'opened']= 0
active_df['opened']=active_df['opened'].astype(int)


In [10]:
active_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 526984 entries, 0 to 526983
Data columns (total 26 columns):
 #   Column                         Non-Null Count   Dtype              
---  ------                         --------------   -----              
 0   event_timestamp                526984 non-null  datetime64[ns, UTC]
 1   event_name                     526984 non-null  object             
 2   user_id                        526984 non-null  object             
 3   document_id                    526984 non-null  object             
 4   surrogate_id                   526984 non-null  object             
 5   created_at                     526984 non-null  datetime64[ns, UTC]
 6   published_at                   526984 non-null  datetime64[ns, UTC]
 7   closed_at                      526984 non-null  datetime64[ns, UTC]
 8   cause                          526984 non-null  object             
 9   effect                         526984 non-null  object             
 10  descript

In [11]:
gpbyincident= active_df.groupby( by= ['document_id', 'description','cause', 'effect'])
gpbyincident2=gpbyincident.sum()
gpbyincident2['count'] = gpbyincident.size()
gpbyincident2['opened_rate'] = gpbyincident2['opened']/gpbyincident2['count']
gpbyincident2



,,,,agency_GewRJAw5tUmC4Ku4AX1-SQ,agency_GtvOEQAFZ0GtU6u4AXwvPg,agency_HE59N3RXM0q5vKu4AXlQZg,agency_JUR9bFXmVkWDHqu4AXaY0g,agency_JfA8Bw8Zp024Kqu4AXiSpQ,agency_MgUq5b9mOEunx6u4AXt_BA,agency_NuuRQ2I1Q0a50Kv-AVKlLA,agency_V2AIQQKgmUO3VazvAOA-Cw,agency_jLjibFoim0iwWau4AWoEdQ,agency_pky7jovXYkaw-awAAMrQ3g,agency_zCy9zG00HEqGeKu4AWZYNQ,notif_reaction,opened,count,opened_rate
document_id,description,cause,effect,,,,,,,,,,,,,,,
-MPvKI-ULauyGmrJiWwV,⏳ ¡Ojo! Retrasos por movilización de cuerpos de emergencia en Calzada México-Tacuba cerca del Metro Cuitláhuac.,EmergencyServices,Delays,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2,0,2,0.000000
-MPvo6hGxeJyKSVcnAUl,✋ ¡Chanclas! Retrasos por accidente sobre Calzada Ignacio Zaragoza cerca del Metro Peñón Viejo hacia la Concordia.,VehicularAccident,Delays,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,3,0,3,0.000000
-MPwL4RRplvymPwZRy2F,⏳ ¡Aguas! Retrasos por percance vehicular en Calz. Ignacio Zaragoza y Río Churubusco.,VehicularAccident,Delays,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,5,0,5,0.000000
-MQCQhzkbRqMo0TsIibZ,"⏳ ¡Aguas! Retrasos por percance vehicular Avenida Cuauhtémoc a la altura de Obrero Mundial, en dirección a Xola.\n\n\n",VehicularAccident,LaneReduction,0.0,0.0,0.0,50.0,50.0,0.0,0.0,0.0,0.0,0.0,0.0,50,4,50,0.080000
-MQCg-TMJBO4h-CsreUU,"¡Ojo! Hay demoras por choque en el Eje 4 Oriente a la altura de la calle Jaime Torres Bodet, hacia el Eje 1 Norte.",VehicularAccident,Delays,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,1,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
-MdyS-3Odath0RqKOve5,⏳ ¡Ojo! Retrasos por percance en Av. Montevideo y Cienfuegos.,VehicularAccident,Delays,0.0,0.0,0.0,0.0,57.0,0.0,0.0,0.0,0.0,0.0,0.0,57,3,57,0.052632
-MdySISY5hSJPgekQ7pR,🚦 ¡Anticípate! Retrasos por semáforos descompuestos en Eje 6 Sur al cruce con Avenida Presidente Plutarco Elías Calles.,BrokenTrafficLight,Delays,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,31.0,31,0,31,0.000000
-MdyjLp8Li6p6HZRc_So,⏳ ¡Úchala! Retrasos por percance en Calzada Ignacio Zaragoza a la altura de Avenida República Federal del Sur con dirección hacia Guelatao.,VehicularAccident,Delays,0.0,94.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,94.0,94,3,94,0.031915


In [12]:
gpbyincident2.describe()

,agency_GewRJAw5tUmC4Ku4AX1-SQ,agency_GtvOEQAFZ0GtU6u4AXwvPg,agency_HE59N3RXM0q5vKu4AXlQZg,agency_JUR9bFXmVkWDHqu4AXaY0g,agency_JfA8Bw8Zp024Kqu4AXiSpQ,agency_MgUq5b9mOEunx6u4AXt_BA,agency_NuuRQ2I1Q0a50Kv-AVKlLA,agency_V2AIQQKgmUO3VazvAOA-Cw,agency_jLjibFoim0iwWau4AWoEdQ,agency_pky7jovXYkaw-awAAMrQ3g,agency_zCy9zG00HEqGeKu4AWZYNQ,notif_reaction,opened,count,opened_rate
count,8331.000000,8331.000000,8331.000000,8331.000000,8331.000000,8331.0,8331.000000,8331.0,8331.000000,8331.000000,8331.000000,8331.000000,8331.000000,8331.000000,8331.000000
mean,0.441604,26.774697,0.883087,20.102629,11.783219,0.0,2.521186,0.0,0.027848,0.072980,36.763294,63.255792,2.912375,63.255792,0.042896
std,8.950607,44.766069,7.467321,82.249478,41.594998,0.0,19.389986,0.0,1.660340,2.143631,48.075617,86.573425,5.231152,86.573425,0.065169
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,4.000000,16.000000,0.000000,16.000000,0.000000
50%,0.000000,1.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,22.000000,37.000000,1.000000,37.000000,0.030000
75%,0.000000,39.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,51.000000,76.000000,3.000000,76.000000,0.060606
max,606.000000,633.000000,227.000000,1246.000000,633.000000,0.0,882.000000,0.0,140.000000,86.000000,882.000000,1246.000000,76.000000,1246.000000,1.000000


In [13]:
gpbycause= active_df.groupby( by= ['cause'])
#gpbyincident= export_df.groupby( by= ['document_id', 'description','cause', 'effect'])
gpbycause2=gpbycause.sum()
gpbycause2['count'] = gpbycause.size()
gpbycause2['opened_rate'] = gpbycause2['opened']/gpbycause2['count']
gpbycause2


,agency_GewRJAw5tUmC4Ku4AX1-SQ,agency_GtvOEQAFZ0GtU6u4AXwvPg,agency_HE59N3RXM0q5vKu4AXlQZg,agency_JUR9bFXmVkWDHqu4AXaY0g,agency_JfA8Bw8Zp024Kqu4AXiSpQ,agency_MgUq5b9mOEunx6u4AXt_BA,agency_NuuRQ2I1Q0a50Kv-AVKlLA,agency_V2AIQQKgmUO3VazvAOA-Cw,agency_jLjibFoim0iwWau4AWoEdQ,agency_pky7jovXYkaw-awAAMrQ3g,agency_zCy9zG00HEqGeKu4AWZYNQ,notif_reaction,opened,count,opened_rate
cause,,,,,,,,,,,,,,,
BrokenTrafficLight,0.0,523.0,31.0,0.0,400.0,0.0,213.0,0.0,0.0,0.0,952.0,961,34,961,0.035380
BrokenVehicle,307.0,11802.0,10.0,0.0,1687.0,0.0,132.0,0.0,0.0,6.0,13529.0,14760,494,14760,0.033469
COVID19,0.0,8.0,0.0,2859.0,41.0,0.0,0.0,0.0,0.0,0.0,131.0,3039,197,3039,0.064824
Counterflow,0.0,0.0,0.0,121.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,121,6,121,0.049587
CycleRide,0.0,468.0,0.0,0.0,216.0,0.0,79.0,0.0,0.0,0.0,516.0,516,15,516,0.029070
Demonstration,1149.0,52615.0,1966.0,5611.0,57058.0,0.0,10740.0,0.0,155.0,211.0,75772.0,114164,5551,114164,0.048623
EmergencyServices,59.0,20429.0,103.0,155.0,2278.0,0.0,454.0,0.0,0.0,83.0,26628.0,27578,956,27578,0.034665
Event,0.0,77.0,0.0,692.0,188.0,0.0,0.0,0.0,0.0,0.0,165.0,965,75,965,0.077720
Explosion,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,14.0,14,3,14,0.214286


In [14]:
gpbyincident2.describe()

,agency_GewRJAw5tUmC4Ku4AX1-SQ,agency_GtvOEQAFZ0GtU6u4AXwvPg,agency_HE59N3RXM0q5vKu4AXlQZg,agency_JUR9bFXmVkWDHqu4AXaY0g,agency_JfA8Bw8Zp024Kqu4AXiSpQ,agency_MgUq5b9mOEunx6u4AXt_BA,agency_NuuRQ2I1Q0a50Kv-AVKlLA,agency_V2AIQQKgmUO3VazvAOA-Cw,agency_jLjibFoim0iwWau4AWoEdQ,agency_pky7jovXYkaw-awAAMrQ3g,agency_zCy9zG00HEqGeKu4AWZYNQ,notif_reaction,opened,count,opened_rate
count,8331.000000,8331.000000,8331.000000,8331.000000,8331.000000,8331.0,8331.000000,8331.0,8331.000000,8331.000000,8331.000000,8331.000000,8331.000000,8331.000000,8331.000000
mean,0.441604,26.774697,0.883087,20.102629,11.783219,0.0,2.521186,0.0,0.027848,0.072980,36.763294,63.255792,2.912375,63.255792,0.042896
std,8.950607,44.766069,7.467321,82.249478,41.594998,0.0,19.389986,0.0,1.660340,2.143631,48.075617,86.573425,5.231152,86.573425,0.065169
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,4.000000,16.000000,0.000000,16.000000,0.000000
50%,0.000000,1.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,22.000000,37.000000,1.000000,37.000000,0.030000
75%,0.000000,39.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,51.000000,76.000000,3.000000,76.000000,0.060606
max,606.000000,633.000000,227.000000,1246.000000,633.000000,0.0,882.000000,0.0,140.000000,86.000000,882.000000,1246.000000,76.000000,1246.000000,1.000000


In [15]:
gpbyuser= active_df.groupby( by= ['user_id'])
#gpbyincident= export_df.groupby( by= ['document_id', 'description','cause', 'effect'])
gpbyuser2=gpbyuser.sum()
gpbyuser2['count'] = gpbyuser.size()
gpbyuser2['opened_rate'] = gpbyuser2['opened']/gpbyuser2['count']
gpbyuser2

,agency_GewRJAw5tUmC4Ku4AX1-SQ,agency_GtvOEQAFZ0GtU6u4AXwvPg,agency_HE59N3RXM0q5vKu4AXlQZg,agency_JUR9bFXmVkWDHqu4AXaY0g,agency_JfA8Bw8Zp024Kqu4AXiSpQ,agency_MgUq5b9mOEunx6u4AXt_BA,agency_NuuRQ2I1Q0a50Kv-AVKlLA,agency_V2AIQQKgmUO3VazvAOA-Cw,agency_jLjibFoim0iwWau4AWoEdQ,agency_pky7jovXYkaw-awAAMrQ3g,agency_zCy9zG00HEqGeKu4AWZYNQ,notif_reaction,opened,count,opened_rate
user_id,,,,,,,,,,,,,,,
00087,0.0,7.0,0.0,0.0,3.0,0.0,1.0,0.0,0.0,0.0,14.0,14,0,14,0.00
00117,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3,0,3,0.00
001e1,0.0,16.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,16.0,16,0,16,0.00
00205,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1,1,1,1.00
0026c,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2.0,2,0,2,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
fff2a,0.0,118.0,0.0,114.0,3.0,0.0,17.0,0.0,0.0,0.0,141.0,248,0,248,0.00
fff48,0.0,14.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,21.0,23,0,23,0.00
fff4d,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4,1,4,0.25


In [16]:
gpbyuser2.describe()

,agency_GewRJAw5tUmC4Ku4AX1-SQ,agency_GtvOEQAFZ0GtU6u4AXwvPg,agency_HE59N3RXM0q5vKu4AXlQZg,agency_JUR9bFXmVkWDHqu4AXaY0g,agency_JfA8Bw8Zp024Kqu4AXiSpQ,agency_MgUq5b9mOEunx6u4AXt_BA,agency_NuuRQ2I1Q0a50Kv-AVKlLA,agency_V2AIQQKgmUO3VazvAOA-Cw,agency_jLjibFoim0iwWau4AWoEdQ,agency_pky7jovXYkaw-awAAMrQ3g,agency_zCy9zG00HEqGeKu4AWZYNQ,notif_reaction,opened,count,opened_rate
count,12343.000000,12343.000000,12343.000000,12343.000000,12343.000000,12343.0,12343.000000,12343.0,12343.000000,12343.000000,12343.000000,12343.000000,12343.000000,12343.000000,12343.000000
mean,0.298064,18.071782,0.596046,13.568419,7.953172,0.0,1.701693,0.0,0.018796,0.049259,24.813660,42.694969,1.965730,42.694969,0.164328
std,0.951865,51.718134,3.919460,31.791810,25.604452,0.0,5.980412,0.0,0.158393,0.353931,65.327312,93.266771,7.483924,93.266771,0.313214
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,1.000000,3.000000,0.000000,3.000000,0.000000
50%,0.000000,2.000000,0.000000,1.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,3.000000,9.000000,0.000000,9.000000,0.000000
75%,0.000000,11.000000,0.000000,10.000000,3.000000,0.0,1.000000,0.0,0.000000,0.000000,18.000000,38.000000,2.000000,38.000000,0.126464
max,17.000000,1275.000000,90.000000,386.000000,384.000000,0.0,135.000000,0.0,4.000000,7.000000,1764.000000,2142.000000,409.000000,2142.000000,1.000000


In [17]:
gpbyuser_dismissers= gpbyuser2.loc[gpbyuser2['opened_rate']== 0 ]
gpbyuser_dismissers

,agency_GewRJAw5tUmC4Ku4AX1-SQ,agency_GtvOEQAFZ0GtU6u4AXwvPg,agency_HE59N3RXM0q5vKu4AXlQZg,agency_JUR9bFXmVkWDHqu4AXaY0g,agency_JfA8Bw8Zp024Kqu4AXiSpQ,agency_MgUq5b9mOEunx6u4AXt_BA,agency_NuuRQ2I1Q0a50Kv-AVKlLA,agency_V2AIQQKgmUO3VazvAOA-Cw,agency_jLjibFoim0iwWau4AWoEdQ,agency_pky7jovXYkaw-awAAMrQ3g,agency_zCy9zG00HEqGeKu4AWZYNQ,notif_reaction,opened,count,opened_rate
user_id,,,,,,,,,,,,,,,
00087,0.0,7.0,0.0,0.0,3.0,0.0,1.0,0.0,0.0,0.0,14.0,14,0,14,0.0
00117,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3,0,3,0.0
001e1,0.0,16.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,16.0,16,0,16,0.0
0026c,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2.0,2,0,2,0.0
002e9,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2,0,2,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ffedf,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4,0,4,0.0
fff2a,0.0,118.0,0.0,114.0,3.0,0.0,17.0,0.0,0.0,0.0,141.0,248,0,248,0.0
fff48,0.0,14.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,21.0,23,0,23,0.0


In [18]:
gpbyuser_openers= gpbyuser2.loc[gpbyuser2['opened_rate']> 0]
gpbyuser_openers

,agency_GewRJAw5tUmC4Ku4AX1-SQ,agency_GtvOEQAFZ0GtU6u4AXwvPg,agency_HE59N3RXM0q5vKu4AXlQZg,agency_JUR9bFXmVkWDHqu4AXaY0g,agency_JfA8Bw8Zp024Kqu4AXiSpQ,agency_MgUq5b9mOEunx6u4AXt_BA,agency_NuuRQ2I1Q0a50Kv-AVKlLA,agency_V2AIQQKgmUO3VazvAOA-Cw,agency_jLjibFoim0iwWau4AWoEdQ,agency_pky7jovXYkaw-awAAMrQ3g,agency_zCy9zG00HEqGeKu4AWZYNQ,notif_reaction,opened,count,opened_rate
user_id,,,,,,,,,,,,,,,
00205,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1,1,1,1.000000
00422,1.0,18.0,0.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0,32.0,32,2,32,0.062500
004d7,0.0,9.0,0.0,0.0,26.0,0.0,5.0,0.0,0.0,0.0,16.0,26,1,26,0.038462
004f5,2.0,12.0,0.0,12.0,0.0,0.0,1.0,0.0,0.0,0.0,11.0,23,2,23,0.086957
00508,1.0,21.0,0.0,68.0,121.0,0.0,4.0,0.0,0.0,0.0,48.0,189,8,189,0.042328
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ffdf2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,2,2,2,1.000000
ffe7c,3.0,11.0,0.0,5.0,3.0,0.0,0.0,0.0,0.0,0.0,28.0,33,1,33,0.030303
ffeea,0.0,18.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,24.0,32,5,32,0.156250


In [19]:
gpbyusera= active_df.groupby( by= ['user_id','document_id', 'description','cause', 'effect'])
#gpbyincident= export_df.groupby( by= ['document_id', 'description','cause', 'effect'])
gpbyusera2=gpbyusera.sum()
gpbyusera2['count'] = gpbyusera.size()
gpbyusera2['opened_rate'] = gpbyusera2['opened']/gpbyusera2['count']
gpbyusera2

0.0   
        -MQwQU5jqKUP8w1ThuXC ⏳ ¡Híjole! Hay retrasos por accidente vehicular... VehicularAccident Delays                                         0.0   
        -MQwWgipOrf6L0SXeEGJ ⏳ ¡changos! Hay retrasos por vehículo descompue... BrokenVehicle     Delays                                         0.0   
...                                                                                                                                              ...   
fff4d   -MSoLbz25XiGvlnd9XGO ⏳ Lánzate a leer el blog de Rumbo porque usuari... Incident          HighWaitingTime                                0.0   
fff82   -MUKNg5tmJaKl5JMYQ9V ✋ ¡Ponte al tiro! Hay cierre de circulación por... Demonstration     CirculationShutdown                            0.0   
        -MWUNCIQud9fDnFKeacc 🚧 ¡Recuerda! Continúa el corte de circulación s... StreetWorks       CirculationShutdown                            0.0   
        -MW_UX1HG-CX1b2YCl0N ✋ ¡Y avanza...! Hay cierre de circulación por c... March             CirculationShutdown                            0.0   
fffef   -MXlm7WGUblRTIJ3eVRB ⏳ ¡No de nuevo! Usuarios informan sobre retraso... Incident          SecuritySpeed                                  0.0   

                                                                                                                       agency_MgUq5b9mOEunx6u4AXt_BA  \
user_id document_id          description                                        cause             effect                                               
00087   -MQXXSmpbOh4k2FoUWLx ⏳ ¡Híjole! Hay retrasos por accidente vehicular... VehicularAccident Delays                                         0.0   
        -MQXz-FFSiWIe8BSujtY ⏳ ¡Ponte trucha! Hay retrasos por accidente veh... VehicularAccident Delays                                         0.0   
        -MQmtcr2MNLKWPIafe51 ⏳ ¡Changos! Hay retrasos por servicios de emerg... EmergencyServices Delays                                         0.0   
        -MQwQU5jqKUP8w1ThuXC ⏳ ¡Híjole! Hay retrasos por accidente vehicular... VehicularAccident Delays                                         0.0   
        -MQwWgipOrf6L0SXeEGJ ⏳ ¡changos! Hay retrasos por vehículo descompue... BrokenVehicle     Delays                                         0.0   
...                                                                                                                                              ...   
fff4d   -MSoLbz25XiGvlnd9XGO ⏳ Lánzate a leer el blog de Rumbo porque usuari... Incident          HighWaitingTime                                0.0   
fff82   -MUKNg5tmJaKl5JMYQ9V ✋ ¡Ponte al tiro! Hay cierre de circulación por... Demonstration     CirculationShutdown                            0.0   
        -MWUNCIQud9fDnFKeacc 🚧 ¡Recuerda! Continúa el corte de circulación s... StreetWorks       CirculationShutdown                            0.0   
        -MW_UX1HG-CX1b2YCl0N ✋ ¡Y avanza...! Hay cierre de circulación por c... March             CirculationShutdown                            0.0   
fffef   -MXlm7WGUblRTIJ3eVRB ⏳ ¡No de nuevo! Usuarios informan sobre retraso... Incident          SecuritySpeed                                  0.0   

                                                                                                                       agency_NuuRQ2I1Q0a50Kv-AVKlLA  \
user_id document_id          description                                        cause             effect                                               
00087   -MQXXSmpbOh4k2FoUWLx ⏳ ¡Híjole! Hay retrasos por accidente vehicular... VehicularAccident Delays                                         0.0   
        -MQXz-FFSiWIe8BSujtY ⏳ ¡Ponte trucha! Hay retrasos por accidente veh... VehicularAccident Delays                                         0.0   
        -MQmtcr2MNLKWPIafe51 ⏳ ¡Changos! Hay retrasos por servicios de emerg... EmergencyServices Delays                                         0.0   
        -MQwQU5jqKUP8w1ThuXC ⏳ ¡Híjole!